<a href="https://colab.research.google.com/github/stuelten/ColorSorter/blob/main/train/train_tensorflow_model_for_arduino.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook shows how to train and convert a TensorFlow model to be used in an Arduino sketch with the EloquentTensorFlow32 or EloquentTensorFlowCortexM libraries.

In [ ]:
! pip install "everywhereml>=0.2.32"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 564.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 2.6 MB/s eta 0:00:00
  Created wheel for everywhereml: filename=everywhereml-0.2.37-py3-none-any.whl size=81193 sha256=be8bdcc366c2dab859db710d3cbae552426a73248f11247056e7ee1f09ea8142
  Stored in directory: /root/.cache/pip/wheels/f0/08/8c/4848a3a49c6b89689c1366cca928db3058dd1f414b03055f15
  Created wheel for hexdump: filename=hexdump-3.3-py3-none-any.whl size=8894 sha256=5c8d2b82d626000f483b28e26d38a2e7afd6229ac385d9b78ea8c9e8cca88668
  Stored in directory: /root/.cache/pip/wheels/26/28/f7/f47d7ecd9ae44c4457e72c8bb617ef18ab332ee2b2a1047e87
  Created wheel for umap-lear

In [ ]:
"""
Get X and y data
(replace with your own)
"""
import numpy as np
from sklearn.datasets import load_iris
from sklearn.preprocessing import MinMaxScaler


def load_Xy():
  """
  Load your data here.
  It may be a CSV file or whatever
  """
  return load_iris(return_X_y=True)


def get_Xy():
  """
  Normalize X
  One-hot encode y
  """
  X, y = load_Xy()
  X = np.asarray(X)
  y = np.asarray(y)
  num_classes = y.max()
  eye = np.eye(num_classes + 1)
  X_norm = MinMaxScaler().fit_transform(X)
  y_hot = np.asarray([eye[yi] for yi in y], dtype=int)

  return X_norm, y_hot

In [ ]:
"""
Instantiate NN for classification
Replace with your own topology
"""
import tensorflow as tf
from tensorflow.keras import layers


def instantiate_nn_for_classification(input_shape, num_classes):
  model = tf.keras.Sequential()
  model.add(layers.Dense(32, activation='relu', input_shape=input_shape))
  model.add(layers.Dense(16, activation='relu'))
  model.add(layers.Dense(num_classes, activation='softmax'))
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

  return model

In [ ]:
"""
Split data between train, validation and test
"""
from sklearn.model_selection import train_test_split


X, y = get_Xy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3)

In [ ]:
"""
Train model
"""
input_shape = X.shape[1:]
num_classes = y.shape[1]

model = instantiate_nn_for_classification(input_shape, num_classes)
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_val, y_val))

Epoch 1/50
5/5 [==============================] - 4s 205ms/step - loss: 1.0825 - accuracy: 0.5068 - val_loss: 1.0709 - val_accuracy: 0.6562
Epoch 2/50
5/5 [==============================] - 0s 42ms/step - loss: 1.0682 - accuracy: 0.5479 - val_loss: 1.0537 - val_accuracy: 0.6875
Epoch 3/50
5/5 [==============================] - 0s 30ms/step - loss: 1.0533 - accuracy: 0.6849 - val_loss: 1.0388 - val_accuracy: 0.7188
Epoch 4/50
5/5 [==============================] - 0s 35ms/step - loss: 1.0395 - accuracy: 0.6986 - val_loss: 1.0250 - val_accuracy: 0.7188
Epoch 5/50
5/5 [==============================] - 0s 26ms/step - loss: 1.0296 - accuracy: 0.6986 - val_loss: 1.0128 - val_accuracy: 0.7188
Epoch 6/50
5/5 [==============================] - 0s 20ms/step - loss: 1.0183 - accuracy: 0.6986 - val_loss: 1.0007 - val_accuracy: 0.7188
Epoch 7/50
5/5 [==============================] - 0s 18ms/step - loss: 1.0065 - accuracy: 0.6986 - val_loss: 0.9880 - val_accuracy: 0.7188
Epoch 8/50
5/5 [==========

In [ ]:
"""
Export NN to C++
Copy-paste the generated code inside a file named model.h or irisModel.h
in your Arduino project
"""
from everywhereml.code_generators.tensorflow import convert_model


c_header = convert_model(model, X, y, model_name='irisModel')
print(c_header)

#pragma once

#ifdef __has_attribute
#define HAVE_ATTRIBUTE(x) __has_attribute(x)
#else
#define HAVE_ATTRIBUTE(x) 0
#endif
#if HAVE_ATTRIBUTE(aligned) || (defined(__GNUC__) && !defined(__clang__))
#define DATA_ALIGN_ATTRIBUTE __attribute__((aligned(4)))
#else
#define DATA_ALIGN_ATTRIBUTE
#endif

// automatically configure network
#define TF_NUM_INPUTS 4
#define TF_NUM_OUTPUTS 3
#define TF_NUM_OPS 2
#define TF_OP_SOFTMAX
#define TF_OP_FULLYCONNECTED




// sample data
float x0[4] = {0.22222222222f, 0.62500000000f, 0.06779661017f, 0.04166666667f};
float x1[4] = {0.75000000000f, 0.50000000000f, 0.62711864407f, 0.54166666667f};
float x2[4] = {0.55555555556f, 0.54166666667f, 0.84745762712f, 1.00000000000f};


/** model size = 5048 bytes **/
const unsigned char irisModel[] DATA_ALIGN_ATTRIBUTE = { 0x1c, 0x00, 0x00, 0x00, 0x54, 0x46, 0x4c, 0x33, 0x14, 0x00, 0x20, 0x00, 0x1c, 0x00, 0x18, 0x00, 0x14, 0x00, 0x10, 0x00, 0x0c, 0x00, 0x00, 0x00, 0x08, 0x00, 0x04, 0x00, 0x14, 0x00, 0x00, 0x00, 0x1c,